In [1]:
# Set up
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('lab04').getOrCreate()

22/01/28 20:36:47 WARN Utils: Your hostname, sheepb-HP-Pavilion-Notebook resolves to a loopback address: 127.0.1.1; using 192.168.1.13 instead (on interface wlo1)
22/01/28 20:36:47 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/01/28 20:36:58 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


### 1. Đọc files và tiền xử lý dữ liệu

#### Đọc files

In [2]:
!ls Lab04-Data/

mushrooms.csv  orders.csv  plants.rar  products.csv


In [3]:
data_path = './Lab04-Data/'

import rarfile
import pandas as pd

#extract file rar
r = rarfile.RarFile(data_path + 'plants.rar')
r.extractall()
r.close()

# đọc file stateabbr.txt : chứa thông tin mô tả về các khu vực được đề cập đến trong tập dữ liệu.
# sử dụng spark.read.text("file_name"): đọc tệp văn bản và trả về dưới dạng cấu trúc dữ SparkDataFrame
# mỗi hàng trong S.DataFrame có giá trịtương ứng với mỗi dòng dữ liệu trong file hay giá trị (value)
info_states = spark.read.text("stateabbr.txt")
    
# đọc files dữ liệu:
#sử dụng pd.read_fwf() để load DataFrames từ các tệp dữ liệu plants: 
# pd.read_fwwf() : Đọc tệp từ đường dẫn đầu vào dưới dạng bảng gồm các dòng được định dạng có độ rộng cố định.
# Các cột thuộc tính trong tệp văn bản được phân tách với chiều rộng cố định nên nội dung DataFrames gồm các cột thuộc tính riêng biệt tương ứng với file_input. 

# đọc  file _names
raw_pl_names = pd.read_fwf('plants.names', encoding='cp1252')

#đọc file _data
raw_pl_data = pd.read_fwf('plants.data', header=None, names=["raw_data"], encoding='cp1252')

In [4]:
raw_pl_names

,Title: Plants
0,Abstract: Data has been extracted from the USD...
1,contains all plants (species and genera) in th...
2,of USA and Canada where they occur.
3,----------------------------------------------...
4,Data Set Characteristics: Multivariate
5,Attribute Characteristics: Categorical
6,Associated Tasks: Clustering
7,Number of Instances: 22632
8,Number of Attributes: 70
9,Missing Values? Yes


In [5]:
raw_pl_data

,raw_data
0,"abelia,fl,nc"
1,"abelia x grandiflora,fl,nc"
2,"abelmoschus,ct,dc,fl,hi,il,ky,la,md,mi,ms,nc,s..."
3,"abelmoschus esculentus,ct,dc,fl,il,ky,la,md,mi..."
4,"abelmoschus moschatus,hi,pr"
...,...
34776,"zuckia brandegeei var. plummeri,co,nm,ut,wy"
34777,"zygophlebia,pr"
34778,"zygophlebia sectifrons,pr"
34779,"zygophyllum,ca,co,id,ks,mt,nv,nm,ny,pa,tx,wa"


In [6]:
#Thông tin mô tả < Tên viết tắt - Tên khu vực>

info_states.show()

+--------------------+
|               value|
+--------------------+
|State Abbreviations |
|                    |
|       U.S. States: |
|          ab Alabama|
|           ak Alaska|
|         ar Arkansas|
|          az Arizona|
|       ca California|
|         co Colorado|
|      ct Connecticut|
|         de Delaware|
|dc District of Co...|
|          fl Florida|
|          ga Georgia|
|           hi Hawaii|
|            id Idaho|
|         il Illinois|
|          in Indiana|
|             ia Iowa|
|           ks Kansas|
+--------------------+
only showing top 20 rows



#### Tiền xử lý

##### Chuyển đổi dữ liệu về dạng vector nhị phân

In [7]:
!pip install sklearn

In [8]:
from sklearn.feature_extraction.text import CountVectorizer

#Lấy danh sách tên các loài 
df_specie = pd.DataFrame( raw_pl_data.raw_data.apply(lambda r: str(r).split(',')[0])).rename(columns = {'raw_data' : 'specie'})
#df_specie = spark.createDataFrame(pd_specie, schema='specie)

#Lấy danh sách các khu vực phân bố của mỗi loài TV
# tương ứng với một chuỗi tên viết tắt các khu vực của loài đó 
df_state = raw_pl_data.raw_data.apply(lambda r: str(r).replace(str(r).split(',')[0] + ',','',1))

# Mã hóa vector biểu diễn khu vực phân bố của mỗi loài
# Sử dụng CountVectorizer() : Chuyển đổi từng khu vực trong mỗi chuỗi dánh sách khu vực, 
#bằng cách tạo một ma trận thưa, trong đó:
# cột ma trận là tên các khu vực địa lý (duy nhất) có trong tập tin dữ liệu,
# hàng là thông tin phân bố khu vực địa lý
# giá trị mỗi ô : 1 hoặc 0. 1 nếu tên khu vực xuất hiện trong chuỗi danh sách phân bố của loài cụ thể, ngược lại bằng 0.

vectorizer = CountVectorizer()
X = vectorizer.fit_transform(df_state)

# Tên các khu vực xuất hiện trong dữ liệu
state = list(vectorizer.get_feature_names_out())

# Ma trận biểu diễn thông tin phân bố đại lý của mỗi loài
df_enc_state = pd.DataFrame(X.toarray(), columns = state)
#df_state =spark.createDataFrame(df_state,schema=state)

# DataFrame thể hiện đầy đủ thông tin phân bố của mỗi loài : <tên loài><thông tin phân bố>
df_plant = spark.createDataFrame( df_specie.join(df_enc_state))

In [9]:
df_plant.show()

22/01/28 20:37:14 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.
22/01/28 20:37:14 WARN TaskSetManager: Stage 1 contains a task of very large size (1441 KiB). The maximum recommended task size is 1000 KiB.


+--------------------+---+---+---+---+---+---+---+---+---+---+---+---+----+-----+---+---+---+----+-----+------+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+
|              specie| ab| ak| al| ar| az| bc| ca| co| ct| dc| de|den|deng|dengl| fl| fr|fra|fras|frasp|fraspm| ga| gl| hi| ia| id| il| in| ks| ky| la| lb| ma| mb| md| me| mi| mn| mo| ms| mt| nb| nc| nd| ne| nf| nh| nj| nm| ns| nt| nu| nv| ny| oh| ok| on| or| pa| pe| pr| qc| ri| sc| sd| sk| tn| tx| ut| va| vi| vt| wa| wi| wv| wy| yt|
+--------------------+---+---+---+---+---+---+---+---+---+---+---+---+----+-----+---+---+---+----+-----+------+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+-

In [10]:
df_plant.printSchema()

root
 |-- specie: string (nullable = true)
 |-- ab: long (nullable = true)
 |-- ak: long (nullable = true)
 |-- al: long (nullable = true)
 |-- ar: long (nullable = true)
 |-- az: long (nullable = true)
 |-- bc: long (nullable = true)
 |-- ca: long (nullable = true)
 |-- co: long (nullable = true)
 |-- ct: long (nullable = true)
 |-- dc: long (nullable = true)
 |-- de: long (nullable = true)
 |-- den: long (nullable = true)
 |-- deng: long (nullable = true)
 |-- dengl: long (nullable = true)
 |-- fl: long (nullable = true)
 |-- fr: long (nullable = true)
 |-- fra: long (nullable = true)
 |-- fras: long (nullable = true)
 |-- frasp: long (nullable = true)
 |-- fraspm: long (nullable = true)
 |-- ga: long (nullable = true)
 |-- gl: long (nullable = true)
 |-- hi: long (nullable = true)
 |-- ia: long (nullable = true)
 |-- id: long (nullable = true)
 |-- il: long (nullable = true)
 |-- in: long (nullable = true)
 |-- ks: long (nullable = true)
 |-- ky: long (nullable = true)
 |-- la: long

#### Lưu dữ liệu đã tiền xử lý

In [11]:
df_plant.write.option( 'header', True).csv(data_path + '/plants.csv')

22/01/28 20:37:15 WARN TaskSetManager: Stage 2 contains a task of very large size (1441 KiB). The maximum recommended task size is 1000 KiB.


In [12]:
!ls ./Lab04-Data/

mushrooms.csv  orders.csv  plants.csv  plants.rar  products.csv


### 2. Kmean

Tất cả các thuộc tính đang được xem xét đều là số hoặc số rời rạc (numeric), do đó chúng ta cần chuyển đổi chúng thành các features bằng cách sử dụng Vector Assembler (transformer).

Vector Assembler(): chuyển đổi một tập hợp các cột features thành một một mảng/vectơ các features.

In [13]:
from pyspark.ml.feature import VectorAssembler

# feature engineering
inputCols = df_plant.columns[1:] # lấy các cột làm đầu vào cho vectorassembler (ngoại trừ cột tên loài)
feature_ass = VectorAssembler(inputCols = inputCols, outputCol = "features") # chuyển các cột input vào cho VTAssembler
data = feature_ass.transform(df_plant) # Transform- - chuyển đổi các cột input thành một vector (mảng) duy nhất

In [14]:
data.show()

22/01/28 20:37:18 WARN TaskSetManager: Stage 3 contains a task of very large size (1441 KiB). The maximum recommended task size is 1000 KiB.


+--------------------+---+---+---+---+---+---+---+---+---+---+---+---+----+-----+---+---+---+----+-----+------+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+--------------------+
|              specie| ab| ak| al| ar| az| bc| ca| co| ct| dc| de|den|deng|dengl| fl| fr|fra|fras|frasp|fraspm| ga| gl| hi| ia| id| il| in| ks| ky| la| lb| ma| mb| md| me| mi| mn| mo| ms| mt| nb| nc| nd| ne| nf| nh| nj| nm| ns| nt| nu| nv| ny| oh| ok| on| or| pa| pe| pr| qc| ri| sc| sd| sk| tn| tx| ut| va| vi| vt| wa| wi| wv| wy| yt|            features|
+--------------------+---+---+---+---+---+---+---+---+---+---+---+---+----+-----+---+---+---+----+-----+------+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---

Huấn luyện model Kmean


In [15]:
from pyspark.ml.clustering import KMeans
from pyspark.ml.evaluation import ClusteringEvaluator

# giá trị thực nghiệm của K
K = range(2,10)
k_score = []

# Kmean model
for k in K:
    print('\n==========K = ' + str(k) + '=============\n')
    # load và train mô hình
    kmeans = KMeans().setK(k).setSeed(1) # .setK( arg = k : số phân cụm), .setSeed(1): cho cùng một giá trị bắt đầu trong mỗi lần gọi random

    model = kmeans.fit(data)
    
    # huấn luyên model trên tập dữ liệu train
    output = model.transform(data)

    
    # Đánh giá mô hình:
    # Đánh giá khả năng phân nhóm - clustering score bằng clusteringEvaluator
    eval = ClusteringEvaluator()
    score = eval.evaluate(output)
    k_score.append(score)
    print("score = " + str(score))

    # center = []
    # for c in model.clusterCenters():
    #     center.append(c)

    # print('\nCluster Center: \n', center)


==========K = 2=============



22/01/28 20:37:19 WARN TaskSetManager: Stage 4 contains a task of very large size (1441 KiB). The maximum recommended task size is 1000 KiB.
22/01/28 20:37:21 WARN TaskSetManager: Stage 5 contains a task of very large size (1441 KiB). The maximum recommended task size is 1000 KiB.
22/01/28 20:37:21 WARN TaskSetManager: Stage 6 contains a task of very large size (1441 KiB). The maximum recommended task size is 1000 KiB.
22/01/28 20:37:21 WARN TaskSetManager: Stage 7 contains a task of very large size (1441 KiB). The maximum recommended task size is 1000 KiB.
22/01/28 20:37:22 WARN TaskSetManager: Stage 8 contains a task of very large size (1441 KiB). The maximum recommended task size is 1000 KiB.
22/01/28 20:37:22 WARN TaskSetManager: Stage 9 contains a task of very large size (1441 KiB). The maximum recommended task size is 1000 KiB.
22/01/28 20:37:22 WARN TaskSetManager: Stage 10 contains a task of very large size (1441 KiB). The maximum recommended task size is 1000 KiB.
22/01/28 20:

score = 0.7163664052872065

==========K = 3=============



22/01/28 20:37:28 WARN TaskSetManager: Stage 34 contains a task of very large size (1441 KiB). The maximum recommended task size is 1000 KiB.
22/01/28 20:37:29 WARN TaskSetManager: Stage 35 contains a task of very large size (1441 KiB). The maximum recommended task size is 1000 KiB.
22/01/28 20:37:29 WARN TaskSetManager: Stage 36 contains a task of very large size (1441 KiB). The maximum recommended task size is 1000 KiB.
22/01/28 20:37:29 WARN TaskSetManager: Stage 37 contains a task of very large size (1441 KiB). The maximum recommended task size is 1000 KiB.
22/01/28 20:37:29 WARN TaskSetManager: Stage 38 contains a task of very large size (1441 KiB). The maximum recommended task size is 1000 KiB.
22/01/28 20:37:29 WARN TaskSetManager: Stage 39 contains a task of very large size (1441 KiB). The maximum recommended task size is 1000 KiB.
22/01/28 20:37:29 WARN TaskSetManager: Stage 40 contains a task of very large size (1441 KiB). The maximum recommended task size is 1000 KiB.
22/01/

score = 0.5783454170344543

==========K = 4=============



...

score = 0.45248246547375737

==========K = 8=============



22/01/28 20:37:49 WARN TaskSetManager: Stage 314 contains a task of very large size (1441 KiB). The maximum recommended task size is 1000 KiB.
22/01/28 20:37:49 WARN TaskSetManager: Stage 315 contains a task of very large size (1441 KiB). The maximum recommended task size is 1000 KiB.
22/01/28 20:37:49 WARN TaskSetManager: Stage 316 contains a task of very large size (1441 KiB). The maximum recommended task size is 1000 KiB.
22/01/28 20:37:49 WARN TaskSetManager: Stage 317 contains a task of very large size (1441 KiB). The maximum recommended task size is 1000 KiB.
22/01/28 20:37:49 WARN TaskSetManager: Stage 318 contains a task of very large size (1441 KiB). The maximum recommended task size is 1000 KiB.
22/01/28 20:37:49 WARN TaskSetManager: Stage 319 contains a task of very large size (1441 KiB). The maximum recommended task size is 1000 KiB.
22/01/28 20:37:49 WARN TaskSetManager: Stage 320 contains a task of very large size (1441 KiB). The maximum recommended task size is 1000 KiB.

score = 0.463012603953447

==========K = 9=============



22/01/28 20:37:51 WARN TaskSetManager: Stage 364 contains a task of very large size (1441 KiB). The maximum recommended task size is 1000 KiB.
22/01/28 20:37:52 WARN TaskSetManager: Stage 365 contains a task of very large size (1441 KiB). The maximum recommended task size is 1000 KiB.
22/01/28 20:37:52 WARN TaskSetManager: Stage 366 contains a task of very large size (1441 KiB). The maximum recommended task size is 1000 KiB.
22/01/28 20:37:52 WARN TaskSetManager: Stage 367 contains a task of very large size (1441 KiB). The maximum recommended task size is 1000 KiB.
22/01/28 20:37:52 WARN TaskSetManager: Stage 368 contains a task of very large size (1441 KiB). The maximum recommended task size is 1000 KiB.
22/01/28 20:37:52 WARN TaskSetManager: Stage 369 contains a task of very large size (1441 KiB). The maximum recommended task size is 1000 KiB.
22/01/28 20:37:52 WARN TaskSetManager: Stage 370 contains a task of very large size (1441 KiB). The maximum recommended task size is 1000 KiB.

score = 0.18946247259197238


##### Kết quả thử nghiệm với K = [  2, 3, 4, ...9 ]

In [16]:
k_score

[0.7163664052872065,
 0.5783454170344543,
 0.5225292282481627,
 0.5014049331510557,
 0.4383148217887277,
 0.45248246547375737,
 0.463012603953447,
 0.18946247259197238]